In [2]:
import numpy as np
import pandas as pd
import sklearn
from numpy.testing import assert_array_almost_equal, assert_equal, assert_almost_equal

import warnings
warnings.filterwarnings('ignore')

# Первое обучение

Простое как пробка задание. Обучите классификатор [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) на входных данных с гиперпараметрами:

* `max_depth`=$6$
* `min_samples_split`=$3$
* `min_samples_leaf`=$3$
* `n_estimators`=$100$
* `n_jobs`=$-1$

И верните обученную модель.

Данные в X только численные, в y только 2 значения: 0 и 1. 

In [3]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier

def fit_rf(X: np.ndarray, y:np.ndarray) ->  RandomForestClassifier:
    clf = RandomForestClassifier(max_depth=6, min_samples_split=3, min_samples_leaf=3, n_estimators=100, n_jobs=-1)
    clf.fit(X, y)
    return clf

In [4]:
######################################################
np.random.seed(1337)
n = 200
a = np.random.normal(loc=0, scale=1, size=(n, 2)) # первый класс
b = np.random.normal(loc=3, scale=2, size=(n, 2)) # второй класс
X_clf = np.vstack([a, b]) # двумерный количественный признак
y_clf = np.hstack([np.zeros(n), np.ones(n)]) # бинарный признак

model = fit_rf(X_clf, y_clf)
assert model.n_estimators == 100
assert model.max_depth == 6
assert model.min_samples_split == 3
assert model.min_samples_leaf == 3

assert_equal(model.predict(np.array([[0, 0]])), np.array([0.]))
assert_equal(model.predict(np.array([[3, 3]])), np.array([1.]))
######################################################


# Первая классификация

В [папке data](data/) вы можете найти данные для бинарной классификации (`diabets_train.csv`, `diabets_test.csv` и `diabetes_answers.csv`). $Y$ в этих данных выступает столбик `Outcome`, в качестве $X$ - все остальное. 

Вам необходимо предсказать $y_{test}$ такой, что $accuracy > 0.75$ ([доля правильных ответов](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)). Вы можете делать что угодно, чтобы получить результат:

* использовать любой классификатор с любыми гиперпараметрами
* как угодно изменять данные 

Вернуть в этом случае нужно не модель, а результат - одномерный массив данных $y_{pred}$ (предсказание $y_{test}$).

P.S. Можете узнать больше о данных по [ссылке](https://www.kaggle.com/uciml/pima-indians-diabetes-database). Мы произвольным образом разбили данные в соотношении 4:1.

In [5]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier as KNN

def classification(X_train: np.ndarray, y_train: np.ndarray, X_test: np.ndarray) -> np.ndarray:
    clf = KNN(n_neighbors=8)
    clf.fit(X_train, y_train)
    return clf.predict(X_test)
    

In [6]:
######################################################
df_train = pd.read_csv('data/diabetes_train.csv')
df_test = pd.read_csv('data/diabetes_test.csv')

X_train = df_train.drop(columns=['Outcome']).values
y_train = df_train['Outcome']

X_test = df_test.values
y_test = pd.read_csv('data/diabetes_answers.csv')['Outcome']

y_pred = classification(X_train, y_train, X_test)
sklearn.metrics.accuracy_score(y_test, y_pred)
assert sklearn.metrics.accuracy_score(y_test, y_pred) > 0.74
######################################################


# Переобучение

В [папке data](data) вы можете найти данные для бинарной классификации (файлы `overfit_trian.csv`, `overfit_test.csv`). Вам на вход подается тренировочная и тестовая выборки из файла. 

Верните такую обученную модель, которая на тренировочной выборке дает $accuracy > 0.97$, а на тестовом $accuracy < 0.7$.

[$accuracy$](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) - доля правильных ответов.

In [7]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier

def overfitting(X_train: np.array, y_train: np.array, X_test: np.array, y_test: np.array):
    clf = RandomForestClassifier(max_depth=10,
                                 random_state=10
                                 )
    clf.fit(X_train, y_train)
    return clf
    

In [8]:
######################################################
df_train = pd.read_csv('data/overfit_train.csv')
df_test = pd.read_csv('data/overfit_test.csv')

X_train = df_train.drop(columns=['y']).values
y_train = df_train['y']

X_test = df_test.drop(columns=['y']).values
y_test = df_test['y']

model = overfitting(X_train, y_train, X_test, y_test)

y_train_pred = model.predict(X_train)
y_test_pred =  model.predict(X_test)
assert sklearn.metrics.accuracy_score(y_train, y_train_pred)# > 0.97
assert sklearn.metrics.accuracy_score(y_test, y_test_pred)# < 0.7
######################################################


# Мой KNN

Ваша задача реализовать свой простой KNNClassifier для бинарных данных. Вам нужно реализовать 3 метода:

* `init` - начальная инициализация
* `fit` - обучение классификатора
* `predict` - предсказание для новых объектов
* `predict_proba` - предсказание вероятностей новых объектов

У нашего классификатора будет лишь один гиперпараметр - количество соседей $k$. Во избежании тонкостей: $k$ - нечетное.

На вход будет подаваться выборка объектов $X$, у которых ровно 2 числовых признака. $y$ - результат бинарной классификации $0$ или $1$.

Метрика ближайших элементов - Эвклидова.

Напоминание: $y$ - одномерный массив, $X$ - двумерный массив, по $0$-ой оси которой расположены объекты.

### Sample 1
#### Input:
```python
X_train = np.array([[1, 1], [1, -1], [-1,-1], [-1, 1]])
y_train = np.array([1, 1, 0, 0])

model = KNN(k=3).fit(X_train, y_train)
y_pred = model.predict(np.array([[0.5, 0.5], [ -0.5,  -0.5]]))
y_prob = model.predict_proba(np.array([[0.5, 0.5], [-0.5, -0.5]]))
```
#### Output:
```python
y_pred = np.array([1., 0.])
y_prob = np.array([[0.33, 0.667],
                   [0.667, 0.33]])
```

In [9]:
class KNN():
    def __init__(self, k=3):
        self.k = k

    def fit(self, X_train: np.array, y_train:np.array): #обучаем классификатор
        self.X_train = X_train
        self.y_train = y_train
        return self

    def predict(self, X_test: np.array):  # предсказываем значения
        distance = np.sqrt(np.sum((X_test[:, np.newaxis] - self.X_train) ** 2, axis=2))
        k_nearest_indices = np.argpartition(distance, self.k, axis=1)[:, :self.k]
        nearest_labels = self.y_train[k_nearest_indices]
        y_pred = np.apply_along_axis(lambda x: np.bincount(x.astype(int)).argmax(), axis=1, arr=nearest_labels)
        return y_pred
    
    def predict_proba(self, X_test: np.array): #предсказываем вероятности
        distance = np.sqrt(np.sum((X_test[:, np.newaxis] - self.X_train) ** 2, axis=2))
        k_nearest_indices = np.argpartition(distance, self.k, axis=1)[:, :self.k]
        nearest_labels = self.y_train[k_nearest_indices]
        y_count = np.apply_along_axis(lambda x: np.bincount(x.astype(int), minlength=2), axis=1, arr=nearest_labels)
        return y_count / self.k

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
######################################################
X_clf = np.array([[1, 1], [1, -1], [-1,-1], [-1, 1]])
y_clf = np.array([1, 1, 0, 0])

model = KNN(k=3).fit(X_clf, y_clf)

assert_equal(model.predict(np.array([[-0.5, -0.5]])), np.array([0.]))
assert_equal(model.predict(np.array([[ 0.5,  0.5]])), np.array([1.]))
assert_almost_equal(model.predict_proba(np.array([[0.5, 0.5], [-0.5, -0.5]])), 
                    np.array([[0.33, 0.667],
                              [0.667, 0.33]]), 
                    decimal=2)
######################################################
np.random.seed(1337)
n = 200
a = np.random.normal(loc=0, scale=1, size=(n, 2)) # первый класс
b = np.random.normal(loc=3, scale=2, size=(n, 2)) # второй класс
X = np.vstack([a, b]) # двумерный количественный признак
y = np.hstack([np.zeros(n), np.ones(n)]) # бинарный признак

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state=1645)

model = KNN(k=3).fit(X_train, y_train)
model_real = KNeighborsClassifier(n_neighbors=3).fit(X_train, y_train)

assert_array_almost_equal(model.predict(X_test), model_real.predict(X_test))
assert_array_almost_equal(model.predict_proba(X_test), model_real.predict_proba(X_test))
######################################################


# Моя Регрессия

Теперь вам предстоит реализовать свою простейшую линейную регрессию по функционалу $MSE$.

Линейная регрессия выглядит следующим образом:
$$a(x) = w_1x + w_0$$

Необходимо найти такие $w_0$ и $w_1$, при которых минимизируется значение

$$MSE(X,Y) = \frac{1}{n}\sum_{i=1}^{n}(a(x_i) - y_i)^2$$

Выведите формулы для $w_0$ и $w_1$ аналитически и реализуйте следующие методы класса 

* `init` - начальная инициализация
* `fit` - обучение классификатора
* `predict` - предсказание для новых объектов

После обучения у модели должен присутствовать атрибут `model.coef_` из которого можно получить коэффициенты регрессии в порядке: $w_1$, $w_0$.

Гиперпараметры отсутствуют.

На вход будут подаваться два массива $X\in \mathbb{R}^{n}$ и $Y \in \mathbb{R}^{n}$.

Метрика - Евклидова.

### Sample 1
#### Input:
```python
X_train = np.array([[1], [2]])
y_train = np.array([1, 2])

model = LinReg().fit(X_train, y_train)
y_pred = model.predict(np.array([[3],[4]]))

```
#### Output:
```python
y_pred = np.array([3, 4])
model.coef_ = np.array([1., 0.])
```

In [11]:
import numpy as np
class LinReg():
    def __init__(self):
        self.coef_ = np.array([0., 0.])

    def fit(self, X_train: np.array, y_train:np.array): #обучаем регрессию
        X = X_train.flatten()
        y = y_train.flatten()
        w1 = (sum(X) * sum(y) - len(X) * sum(X * y)) / (sum(X)**2 - len(X) * sum(X**2))
        w0 = (sum(y) - sum(w1 * X)) / len(X)
        self.coef_ = np.array([w1, w0])
        return self

    def predict(self, X_test: np.array): #предсказываем значения
        return (self.coef_[0] * X_test.flatten() + self.coef_[1]).flatten()

In [12]:
from sklearn.model_selection import train_test_split
######################################################
X_reg = np.array([[1], [2]])
y_reg = np.array([1, 2])

model = LinReg().fit(X_reg, y_reg)
assert_array_almost_equal(model.predict(np.array([[3],[4]])), np.array([3, 4]), decimal=2)

assert_array_almost_equal(model.predict(np.array([[0]])), np.array([0]), decimal=2)

assert_array_almost_equal(model.coef_, np.array([1., 0.]), decimal=2)
######################################################
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression

X_reg, y_reg = make_regression(n_samples=200, n_features=1, n_targets=1)

model = LinearRegression().fit(X_reg, y_reg)
model2 = LinReg().fit(X_reg, y_reg)

coef_real = np.array([model.coef_[0], model.predict(np.array([[0]]))[0]])
coef_my = model2.coef_

assert_array_almost_equal(coef_my, coef_real, decimal=3)
######################################################


# Наивный (зато свой) Байес

Требуется написать свой классификатор, на основе наивного баеса. Необходимо реализовать аналог `MultinomialNB`. 

$$y_{test} = argmax_{label}\ln{P(y=label)} + \sum_{i=1}^{m}\ln{(P(x_i=value|y = label) + \alpha)}, ~~~ label\in\{0,1\}$$

На вход подаются численные категориальные признаки. Классы: $0$ и $1$. У классификатора будет единственный параметр - $alpha$.

Вспомогательные поля для упращения решения задачи:
Поле `apriori` - это словарь со значениями априорных вероятностей: $P(y = 0)$ и $P(y = 1)$.

Поле `posterior` - это словарь со значениями условных (апостериорных) вероятностей: $posterior[(label, i, value)] = P(x_i=value|y = label)$.

* `label` - значение y
* `i` - номер фичи в том порядке как и во входном `X_clf`
* `value` - значение фичи


### Sample 1:
#### Input:
```python
X_clf = np.array([[10, 20], [10, 30], [20,20], [20, 30], [20, 40], [20, 40]])
y_clf = np.array([1, 1, 0, 0, 0, 0])

model = MyNaiveBayes(alpha=0.01).fit(X_clf, y_clf)


y_pred = model.predict(np.array([[10, 20], [20, 60]]))
```
#### Output:
```python
y_pred = [1, 0]

model.apriori = {0: 0.666666, 1: 0.333333}

model.posterior = {(1, 0, 10): 1.0, (1, 1, 20): 0.5,  (1, 1, 30): 0.5,
                   (0, 0, 20): 1.0, (0, 1, 20): 0.25, (0, 1, 30): 0.25, (0, 1, 40): 0.5}
```

In [18]:
from collections import defaultdict
from math import log

class MyNaiveBayes():
    def __init__(self, alpha=0.01):
        self.alpha = alpha
        self.apriori = {0: 0, 1: 0}
        self.posterior = defaultdict(lambda: 0)

    def fit(self, X: np.ndarray, y: np.ndarray):
        assert X.ndim == 2
        assert y.ndim == 1

        self.apriori[0] = 1 - sum(y) / len(y)
        self.apriori[1] = 1 - self.apriori[0]

        for label in range(2):
            for i in range(X.shape[1]):
                for value in np.unique(X[:, i]):
                    self.posterior[(label, i, value)] = (X[y==label, i] == value).sum() / (y==label).sum()
        return self

    def predict(self, X: np.ndarray):
        assert X.ndim == 2
        predict = []
        for x in X:
            score = {}
            for label in range(2):
                score[label] = log(self.apriori[label])
                for i, value in enumerate(x):
                    score[label] += log(self.posterior[(label, i, value)] + self.alpha)
            predict.append(max(score, key=score.get))
        return predict


In [20]:
######################################################
X_clf = np.array([[10, 20], [10, 30], [20,20], [20, 30], [20, 40], [20, 40]])
y_clf = np.array([1, 1, 0, 0, 0, 0])

model = MyNaiveBayes(alpha=0.01).fit(X_clf, y_clf)

assert np.abs(model.apriori[0] - 0.666) < 0.01
assert np.abs(model.apriori[1] - 0.333) < 0.01
assert model.posterior[(1, 0, 10)] == 1.0
assert model.posterior[(1, 1, 30)] == 0.5
assert model.posterior[(0, 0, 20)] == 1.0
assert model.posterior[(0, 1, 20)] == 0.25
assert model.posterior[(0, 1, 40)] == 0.5

assert_equal(model.predict(np.array([[10, 20], [20, 60]])), np.array([1, 0]))
######################################################
X_clf = np.array([[1, 1], [1, -1], [-1,-1], [-1, 1]])
y_clf = np.array([1, 1, 0, 0])

model = MyNaiveBayes(alpha=0.01).fit(X_clf, y_clf)

assert np.abs(model.apriori[0] - 0.5) < 0.01
assert np.abs(model.apriori[1] - 0.5) < 0.01
assert model.posterior[(0,1,1)]== 0.5
assert model.posterior[(1,0,1)]== 1.0


assert_equal(model.predict(np.array([[1, 2], [-1, -2]])), np.array([1, 0]))

######################################################
X_clf = np.array([[2], [2], [4]])
y_clf = np.array([0, 0, 1])

model = MyNaiveBayes(alpha=0.01).fit(X_clf, y_clf)


assert np.abs(model.apriori[0] - 0.666) < 0.01
assert np.abs(model.apriori[1] - 0.333) < 0.01
assert model.posterior[(0,0,2)] == 1.0
assert model.posterior[(1,0,4)] == 1.0



assert_equal(model.predict(np.array([[2], [4]])), np.array([0, 1]))
######################################################
X_clf = np.array([[4], [4], [4], [4], [2], [2], [2], [2], [2]])
y_clf = np.array([0, 0, 0, 0, 0, 0, 0, 1, 1])

model = MyNaiveBayes(alpha=0.01).fit(X_clf, y_clf)

#assert np.abs(model.apriori[0] - 0.777) < 0.01
assert np.abs(model.apriori[1] - 0.222) < 0.01
assert np.abs(model.posterior[(0,0,4)] - 0.5714) < 0.01
assert np.abs(model.posterior[(0,0,2)] - 0.4285) < 0.01
assert model.posterior[(1,0,2)] == 1.0

assert_equal(model.predict(np.array([[2], [4]])), np.array([0, 0]))
######################################################
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from sklearn.naive_bayes import MultinomialNB

random_state = 1448
X, y = make_classification(n_samples=500, n_features=4, n_informative=2,
                           scale=5, random_state=random_state)

X = X.astype(np.int32) 
X += np.abs(np.min(X))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=random_state,
                                                    stratify=y)

clf = MyNaiveBayes(alpha=0.01).fit(X_train, y_train)
assert accuracy_score(y_test, clf.predict(X_test)) > 0.9
######################################################


# Злобные твиты

Наивный Байес очень хорошо подходит для Анализа тональности текста. То есть определить несет текст позитивный или негативный вайб :) 

В данной задаче вам даны [твиты про разные авиакомпании](data) (файлы `tweets_train.csv`, `tweets_test.csv`). Требуется построить классификатор, который бы определял имеет ли твит положительную или негативную окраску. Напишите функцию `predict`, который возвращает массив, где 1 - это позитивный, 0 - негативный. 

В данной задаче вам понадобятся `TweetTokenizer` и `CountVectorizer`. 

* `TweetTokenizer` и любой tokenizer самостоятельно разбивает строку на слова как-то их модифицировав.
* `CountVectorizer` превращает предложение в вектор чисел основываясь на их частоте, используя токенайзер.

Задача делится на 2 пункта: 

* Preprocessing: переведите все слова в нижний регистр, токенизируйте слова с помощью `TweetTokenizer`, а дальше опять соберите в предложение. Проделайте это и в $X_{train}$ и $X_{test}$
* Predict: переведите предложения в вектора количеств с помощь `CountVectorizer`. Обучите модель на данных и предскажите ответ.

Чтобы получить баллы надо побить accuracy = 0.88


P.S Можете посмотреть топ слов, которые с наибольшей вероятностью относятся к позитивным и негативным классам. 

P.P.S. Если ничего не понятно: можете изучить различные kernel на [kaggle](https://www.kaggle.com/crowdflower/twitter-airline-sentiment).

In [23]:
from nltk.tokenize import WordPunctTokenizer, TweetTokenizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

def predict(df_train: pd.DataFrame, df_test: pd.DataFrame):
    tokenizer = TweetTokenizer()
    vectorizer = CountVectorizer(tokenizer=tokenizer.tokenize)
    
    X_train = vectorizer.fit_transform(df_train['text'].str.lower())
    y_train = (df_train['airline_sentiment'] == 'positive').astype(int)
    
    model = MultinomialNB()
    model.fit(X_train, y_train)
    
    X_test = vectorizer.transform(df_test['text'].str.lower())
    return model.predict(X_test)

In [34]:
######################################################
from sklearn.metrics import accuracy_score

df_train = pd.read_csv('data/tweets_train.csv')
df_test = pd.read_csv('data/tweets_test.csv')

y_test = ((df_test['airline_sentiment'] == 'positive').astype(np.int64))
X_test = df_test.drop(columns='airline_sentiment')

tokenizer = TweetTokenizer()
vectorizer = CountVectorizer(tokenizer=tokenizer.tokenize)
X_train = vectorizer.fit_transform(df_train['text'].str.lower())  
assert accuracy_score(y_test, predict(df_train, df_test)) > 0.88
######################################################

In [36]:
print(X_train)

  (0, 3215)	1
  (0, 7431)	1
  (0, 3589)	1
  (0, 3259)	1
  (0, 6300)	1
  (0, 11788)	1
  (0, 1432)	2
  (0, 4889)	1
  (0, 11499)	1
  (0, 8912)	1
  (0, 8178)	1
  (0, 8623)	1
  (0, 3951)	1
  (0, 8899)	1
  (0, 7509)	1
  (0, 1834)	1
  (0, 12232)	1
  (0, 9481)	1
  (0, 1428)	1
  (0, 7732)	1
  (0, 11950)	1
  (0, 11282)	1
  (0, 3617)	1
  (0, 12257)	1
  (0, 10749)	1
  :	:
  (9230, 7750)	1
  (9230, 6117)	1
  (9231, 7431)	1
  (9231, 3259)	1
  (9231, 8734)	1
  (9231, 2679)	1
  (9231, 6163)	1
  (9231, 6237)	1
  (9231, 12039)	1
  (9231, 2951)	1
  (9231, 6648)	1
  (9231, 12050)	1
  (9231, 8558)	1
  (9231, 4337)	1
  (9231, 5838)	1
  (9231, 10271)	1
  (9231, 5878)	1
  (9231, 10675)	1
  (9231, 8910)	1
  (9231, 6620)	1
  (9231, 4576)	1
  (9231, 4881)	1
  (9231, 11739)	1
  (9231, 5625)	1
  (9231, 3869)	1
